#  Pointcloud web viewer

Hackathon 22-08 - *Ada, Tomas, Mantas*.
Program reads data from .laz file (in Jupyter notebook). Voila creates stand alone web applicatoin to view 3D data. 

In [1]:
import numpy as np
import ipyvolume as ipv
from laspy import CopcReader, Bounds

In [2]:
def create_query(header):
    """
    Function to read copc standart data
    """
    querys = []

    sizes = header.maxs - header.mins

    # Bottom left
    query_bounds = Bounds(
        mins=header.mins,
        maxs=header.mins + sizes / 2
    )
    query_bounds.maxs[2] = header.maxs[2]
    querys.append(query_bounds)

    # Top Right
    # Bounds can also be 2D (then all Z are considered)
    query_bounds = Bounds(
        mins=(header.mins + sizes / 2)[:2],
        maxs=header.maxs[:2]
    )
    querys.append(query_bounds)

    return querys


In [3]:
path = "millsite.copc.laz"

with CopcReader.open(path) as crdr:

    querys = create_query(crdr.header)

    for i, query_bounds in enumerate(querys):
        resolution = None
        points_cloud = crdr.query(query_bounds, resolution=resolution)

X = points_cloud.x
Y = points_cloud.y
Z = points_cloud.z
n = 0  # start points range
m = 400000  # end points range

points = np.array(np.vstack((X[n:m]-np.min(X[n:m]), Y[n:m]-np.min(Y[n:m]),
                                 Z[n:m]-np.min(Z[n:m]))).transpose())

x = points[:, 0]
y = points[:, 1]
z = points[:, 2]

# Plot data
fig = ipv.figure(width = 1000, height=1000)
scatter = ipv.scatter(x, y, z,  marker="sphere")
ipv.show()
